## 03 Feature Extraction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import torch
import time

#### Check GPU availability

In [2]:
from chosen_gpu import get_freer_gpu
device = torch.device(get_freer_gpu()) 
print("Configured device: ", device)

Configured device:  cuda:2


### 1. Loading Data

In [3]:
# par = "par16"
# file = "16-Rom_1_2021-04-09-10.33.02"
# task = "_visual"

# # path = '../data/np/{par}/{file}{task}.npy'.format(par=par,file=file, task=task)

# X = np.load('../data/np/round2/{par}/{file}{task}_X.npy'.format(par=par,file=file, task=task))
# y = np.load('../data/np/round2/{par}/{file}{task}_y.npy'.format(par=par,file=file, task=task))

In [4]:
par = "par1_v2"
file = "1-Chaichan_1_2021-04-07-06.24.14"
task = "_visual"
model_name = "cnn"
drift = "no_drift"
# path = '../data/np/{par}/{file}{task}.npy'.format(par=par,file=file, task=task)

X = np.load('../data/np/round2/{par}/{drift}/{file}{task}_X.npy'.format(par=par,drift = drift,file=file, task=task), allow_pickle=True)
y = np.load('../data/np/round2/{par}/{drift}/{file}{task}_y.npy'.format(par=par,drift = drift,file=file, task=task), allow_pickle=True)

#### 1.1 Check shape

In [5]:
# [# stim, # electrod, # datapoint]
print(X.shape)
print(y.shape)

(150, 16, 49)
(150,)


#### 1.2 Plot

In [6]:
# ### Plot to see wheter eegs have drift or not
# data = X.reshape(-1,16)
# print(data.shape)
# fig, ax = plt.subplots(16,1,figsize=(200,100),sharex=True)

# start_point = 3000
# plot_lenght = 4000

# for i in range(data.shape[1]):
#     ax[i].plot(range(plot_lenght),data[start_point:start_point+plot_lenght,i])

### 2. Split data
- test_size: 0.1
- 10% of data is reserved for the real test --> X_test, y_test
- 90% will be again divided into (train,test,val) --> X_model, y_model

#### 2.1 Reserve some data for REAL TEST

In [7]:
from sklearn.model_selection import train_test_split

X_model, X_test, y_model, y_test = train_test_split( X, y, test_size=0.1, random_state=42)

In [8]:
print("Shape of X_model: ", X_model.shape)
print("Shape of X_test: ",X_test.shape)
print("Shape of y_model: ",y_model.shape)
print("Shape of y_test: ",y_test.shape)

Shape of X_model:  (135, 16, 49)
Shape of X_test:  (15, 16, 49)
Shape of y_model:  (135,)
Shape of y_test:  (15,)


#### 2.2 Chunking

- 10 can be thought of as totally new eeg records and will be used as the real evaluation of our model.
- For X : Chunking eeg to lengh of 10 data point in each stimuli's eeg
- For y(lebels) : Filled the lebels in y because we chunk X ( 1 stimuli into 6 chunk). We have 500 labels before but now we need 500 x 6 = 3000 labels

In [9]:
import sys
np.set_printoptions(threshold=sys.maxsize)

def chunk_data(data, size):
    data_keep = data.shape[2] - (data.shape[2]%size)
    #print(f'{data.shape}')
    data = data[:,:,:data_keep]
    #print(f'{data.shape}')
    #print(data[0,0,:20])
    data = data.reshape(-1,data.shape[1],data.shape[2]//size,size)
    #print(f'{data.shape}')
    #print(data[0,0,:2,:])
    data = np.transpose(data, (0, 2, 1, 3)  )
    #print(f'{data.shape}')
    #print(data[0,:2,0,:])
    return data

def filled_y(y, chunk_num):
    yy = np.array([[i] *chunk_num for i in  y ]).ravel()
    return yy

In [10]:
chunk_size = 10 

print('=================== X ==================')
print(f'Oringinal X shape {X_model.shape}')
X = chunk_data(X_model, chunk_size)
print(f'Chunked X : {X.shape}') # (#stim, #chunks, #electrodes, #datapoint per chunk)
chunk_per_stim = X.shape[1]
X = X.reshape(-1,16,chunk_size)
print(f'Reshape X to : {X.shape}')
print('=================== y ==================')
print(f'Shape of y : {y_model.shape}')

y_new = np.zeros(2430) #100 labels, 62 chunks per label
print(y_new.shape)
size = 10
for i,marker in enumerate(y_model):
#     print(marker)
    y_new[i*size:(i+1)*size] = y_model[i]
y = y_new
print(y)
# print(y_new)
y_filled = filled_y(y_model, chunk_per_stim)
print(y_filled)
print(f'Shape of new y : {y.shape}')

=================== X ==================
Oringinal X shape (135, 16, 49)
Chunked X : (135, 4, 16, 10)
Reshape X to : (540, 16, 10)
=================== y ==================
Shape of y : (135,)
(2430,)
[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

In [11]:
print(y_model)

[2 2 0 1 0 2 1 2 2 2 1 0 2 0 0 1 0 2 2 0 2 0 0 1 2 1 1 2 1 1 0 1 2 0 1 0 0
 0 0 2 1 1 1 2 0 0 0 1 2 1 0 2 0 2 1 2 0 0 2 2 0 1 2 2 0 0 1 2 2 2 1 0 1 1
 1 2 0 2 2 0 2 2 1 2 2 0 2 2 0 1 0 1 1 2 0 0 0 2 1 1 1 1 0 1 2 1 2 1 0 0 1
 2 1 0 0 0 0 1 2 1 1 1 1 0 2 0 2 1 2 2 2 1 2 2 0]


### 3. Define Model

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import fnmatch

In [13]:
class LSTM(nn.Module):
    
    '''
    Expected Input Shape: (batch, seq_len, channels)
    '''
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, bidirectional, dropout):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, bidirectional=bidirectional, 
                            dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * num_layers, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Set initial hidden and cell states
        #*2 because it's bidirectional
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(device).float()
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(device).float()
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0)) # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :]) 
        out = self.softmax(out)

        return

class Conv1D_LSTM(nn.Module):
    '''
    Expected Input Shape: (batch, seq_len, channels)
    '''
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, bidirectional, dropout):
        super(Conv1D_LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.conv_1d = nn.Sequential(nn.Conv1d(input_dim, input_dim, [1,1]),nn.Dropout(dropout))
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, bidirectional=bidirectional, 
                           dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * num_layers, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        # Creating learnable preprocessing using conv1d
        # conv1d expects (batch, channels, seq_len)
        x = x.unsqueeze(0)
        x = x.permute(1,3,0,2)   
        x = self.conv_1d(x)
        x = x.squeeze(2).permute(0,2,1)
        
        # Set initial hidden and cell states
        #*2 because it's bidirectional
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(device).float()
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(device).float()
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0.detach(), c0.detach())) # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :]) 
        out = self.softmax(out)
        
        return out

### 4. Define Training function

Define the training process

We set `model.train()` so dropout is applied.

In [14]:
def train(model, iterator, optimizer, criterion):
    total = 0
    correct = 0
    epoch_loss = 0
    epoch_acc = 0
    
    predicteds = []
    trues = []    
    
    model.train()
    
    for batch, labels in iterator:

        #Move tensors to the configured device
        batch = batch.to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = model(batch.float())
        loss = criterion(outputs, labels.long())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
                
        #check accuracy
        predictions = model(batch.float())
        _, predicted = torch.max(predictions.data, 1)  #returns max value, indices
        predicteds.append(predicted)
        trues.append(labels)        
        total += labels.size(0)  #keep track of total
        correct += (predicted == labels).sum().item()  #.item() give the raw number
        acc = 100 * (correct / total)
                
        epoch_loss += loss.item()
        epoch_acc = acc
        
    return epoch_loss / len(iterator), epoch_acc,predicteds, trues

In [15]:
def evaluate(model, iterator, criterion):
    
    total = 0
    correct = 0
    epoch_loss = 0
    epoch_acc = 0
    
    predicteds = []
    trues = []
    
    model.eval()
    
    with torch.no_grad():
    
        for batch, labels in iterator:
            
            #Move tensors to the configured device
            batch = batch.to(device)
            labels = labels.to(device)

            predictions = model(batch.float())
            loss = criterion(predictions, labels.long())

            _, predicted = torch.max(predictions.data, 1)  #returns max value, indices
            clear_output(wait=True)
            print('================== Predicted y ====================')
            print(predicted) 
            print('==================    True y   ====================')
            print(labels)            
            predicteds.append(predicted)
            trues.append(labels)            
            total += labels.size(0)  #keep track of total
            correct += (predicted == labels).sum().item()  #.item() give the raw number
            acc = 100 * (correct / total)
            
            epoch_loss += loss.item()
            epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator),predicteds, trues

### 5. Actual Training for Feature Extraction 


#### 5.1 Define model parameters
- Count model parameters
- optimizer
- loss function
- GPU

In [16]:
# #Create the model from class
# model_EEGEncoder = EEGEncoder()
# model_EEGEncoder = model_EEGEncoder.float() #define precision as float to reduce running time
# models = [model_EEGEncoder]


# #Count the parameters for writing papers
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

# for model in models:
#     print(f'The model {type(model).__name__} has {count_parameters(model):,} trainable parameters')# Train the model


#### 5.2 Prepare X and y in correct shape

- For X, pytorch (if set batch_first) LSTM requires to be (batch, seq_len, features).  Thus, for us, it should be (100, 75, 16).
- For y, nothing is special
- So let's convert our numpy to pytorch, and then reshape using view

In [17]:
torch_X = torch.from_numpy(X)
torch_y = torch.from_numpy(y)

print("Shape of torch_X: ",torch_X.shape)
print("Shape of torch_y: ",torch_y.shape)

Shape of torch_X:  torch.Size([540, 16, 10])
Shape of torch_y:  torch.Size([2430])


In [18]:
print("Original X: ", torch_X.size())

Original X:  torch.Size([540, 16, 10])


CNN requires the input shape as (batch, channel, height, width)

In [19]:
# torch_X_reshaped = torch_X.reshape(torch_X.shape[0],torch_X.shape[1],1,torch_X.shape[2])
torch_X_reshaped = torch_X.reshape(torch_X.shape[0], torch_X.shape[2], torch_X.shape[1])
print("Converted X to ", torch_X_reshaped.size())

Converted X to  torch.Size([540, 10, 16])


#### 5.3 Split test train set, and load them into a DataLoader

In [20]:
from torch.utils.data import TensorDataset

# Define dataset
ds = TensorDataset(torch_X_reshaped, torch_y)

#Train test split
train_size = int(torch_X_reshaped.size()[0] * 0.7)
valid_size = int(torch_X_reshaped.size()[0] * 0.2)
test_size  = torch_X_reshaped.size()[0] - train_size - valid_size

train_set, valid_set, test_set = torch.utils.data.random_split(ds, [train_size, valid_size, test_size])



BATCH_SIZE = 128 #keeping it binary so it fits GPU
#Train set loader
train_iterator = torch.utils.data.DataLoader(dataset=train_set, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

#Validation set loader
valid_iterator = torch.utils.data.DataLoader(dataset=valid_set, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

#Test set loader
test_iterator = torch.utils.data.DataLoader(dataset=test_set, 
                                          batch_size=test_size, 
                                          shuffle=True)


AssertionError: 

In [ ]:
input_dim = 16
hidden_dim = 32
num_layers = 2
num_classes = 10
bidirectional = True
dropout = 0.65

#CONV1D + LSTM
model_conv1d_lstm = Conv1D_LSTM(input_dim, hidden_dim, num_layers, num_classes, bidirectional, dropout)
model_conv1d_lstm = model_conv1d_lstm.float() #define precision as float to reduce running time

models = [model_conv1d_lstm]

#### 5.4 Train the model

In [ ]:
import torch.optim as optim

best_valid_loss = float('inf')
train_losses    = []
valid_losses    = []

learning_rate = 0.0001
N_EPOCHS      = 5000          ## best is 10k
criterion     = nn.NLLLoss()

for model in models:
    model = model.to(device)
criterion = criterion.to(device)
optimizer     = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
 import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 2000

best_valid_loss = float('inf')

train_losses = []
valid_losses = []

train_accs = []
valid_accs = []

train_predicted_labels = []
valid_predicted_labels = []

train_true_labels = []
valid_true_labels = []


for i, model in enumerate(models):
    print(f"Training {type(model).__name__}")
    
    start_time = time.time()

    for epoch in range(N_EPOCHS):
        start_time = time.time()

        train_loss, train_acc, train_pred_label, train_true_label = train(model, train_iterator, optimizer, criterion)
        valid_loss, valid_acc, valid_pred_label, valid_true_label = evaluate(model, valid_iterator, criterion)
        train_losses.append(train_loss); train_accs.append(train_acc); train_predicted_labels.append(train_pred_label); train_true_labels.append(train_true_label); 
        valid_losses.append(valid_loss); valid_accs.append(valid_acc); valid_predicted_labels.append(valid_pred_label); valid_true_labels.append(valid_true_label); 

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        if (epoch+1) % 5 == 0:
            clear_output(wait=True)            
            print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
            print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc:.2f}%')
            print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc:.2f}%')

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            print("Model:{} saved.".format(type(model).__name__))
            torch.save(model.state_dict(),"../model/feature_extraction/round2/{par}/EEG_ENCODER.pt.tar".format(par=par))
            best_model_index = i

In [ ]:
# model.is_debug = False
# iteration = 0

# for i, model in enumerate(models):
#     print(f"Training {type(model).__name__}")
    
#     start_time = time.time()

#     for epoch in range(N_EPOCHS):
#         start_time = time.time()

#         train_loss, train_acc, train_predicted    = train(model, train_iterator, optimizer, criterion)
#         valid_loss, valid_acc, valid_predicted, _ = evaluate(model, valid_iterator, criterion)

#         train_losses.append(train_loss)
#         valid_losses.append(valid_loss)
        
#         end_time = time.time()

#         epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
#         iteration     += 1
        
#         if (epoch+1) % 50 == 0:
#             clear_output(wait=True)
#             print(f'Epoch: {epoch+1:02}/{N_EPOCHS}  |',end='')
#             print(f'\tTrain Loss: {train_loss:.5f}  | Train Acc: {train_acc:.2f}%  |', end='')
#             print(f'\t Val. Loss: {valid_loss:.5f}  | Val. Acc: {valid_acc:.2f}%')
#             do_plot(train_losses, valid_losses)
          

#         if valid_loss < best_valid_loss:
#             best_valid_loss = valid_loss
#             print("Model:{} saved.".format(type(model).__name__))
#             torch.save(model.state_dict(), "../model/feature_extraction/round2/{par}/EEG_ENCODER.pt.tar".format(par=par))
#             best_model_index = i


### 7. Evaluation (Test model)
using test set

In [ ]:
def squeeze_to_list(_tmp):
    from functools import reduce
    import operator

    xx     = [ i.cpu().detach().numpy().ravel().tolist() for i in _tmp]
    xx     = reduce(operator.concat, xx)
    return xx



models[best_model_index].load_state_dict(torch.load('../model/feature_extraction/round2/{par}/EEG_ENCODER.pt.tar'.format(par=par)))



# test_loss = evaluate(models[best_model_index], test_iterator, criterion)
# print(f'Test Loss: {test_loss}') # | Test Acc: {test_acc:.2f}%')

test_loss, test_acc , predicted, actual_labels = evaluate(models[best_model_index], test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc:.2f}%')
print("---------------")
print(" (Actual y , Predicted y)")

y_test     = squeeze_to_list(actual_labels)
y_hat_test = squeeze_to_list(predicted)

out = zip(y_test, y_hat_test)

print(list(out))



### 8. Save features extracted

In [ ]:

# encoder_best_model = models[best_model_index].load_state_dict(torch.load(f'save/{type(model).__name__}{i}.pth.tar'))
# torch.save(encoder_best_model, 'save/7.1_encoder_best_model.pth.tar')


### save Encoder network
# torch.save(model_EEGEncoder, 'save/model_EEGEncoder_network_5s.pt.tar')
# torch.save(test_iterator,'save/eeg_X_test_5s.pt.tar')


# save extracted features
eeg_encode = model_EEGEncoder.get_latent(torch_X_reshaped.to(device).float())
eeg_extracted_features = eeg_encode.detach().cpu().numpy()
np.save('../data/extracted_features/round2/{par}/{file}{task}_X'.format(par=par,file=file, task=task), eeg_extracted_features )
np.save('../data/extracted_features/round2/{par}/{file}{task}_y'.format(par=par,file=file, task=task),y )